In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import widgets, Layout
from IPython.display import display
from dash import Dash, dcc, html, Input, Output
import os
from threading import Timer
import webbrowser

In [2]:
import warnings
warnings.simplefilter(action="ignore",category=FutureWarning)

In [3]:
def open_browser():
    if not os.environ.get("WERKZEUG_RUN_MAIN"):
        webbrowser.open_new('http://127.0.0.1:8080/')

In [4]:
linkedin_job_postings=pd.read_csv("..\Data\linkedin_jobs_and_skills\linkedin_job_postings_cleaned.csv")

In [5]:
linkedin_job_postings.head()

,job_link,job_title,company,job_location,first_seen,search_city,search_country,search_position,job_level,job_type,current_time,current_date
0,https://www.linkedin.com/jobs/view/account-exe...,account executive - dispensing (norcal/norther...,BD,"San Diego, CA",2024-01-15,Coronado,United States,Color Maker,Mid senior,Onsite,07:12:29.002560,2024-01-21
1,https://www.linkedin.com/jobs/view/registered-...,registered nurse - rn care manager,Trinity Health MI,"Norton Shores, MI",2024-01-14,Grand Haven,United States,Director Nursing Service,Mid senior,Onsite,07:39:58.881370,2024-01-21
2,https://www.linkedin.com/jobs/view/restaurant-...,restaurant supervisor - the forklift,Wasatch Adaptive Sports,"Sandy, UT",2024-01-14,Tooele,United States,Stand-In,Mid senior,Onsite,07:40:00.251126,2024-01-21
3,https://www.linkedin.com/jobs/view/independent...,independent real estate agent,Howard Hanna | Rand Realty,"Englewood Cliffs, NJ",2024-01-16,Pinehurst,United States,Real-Estate Clerk,Mid senior,Onsite,07:40:00.308133,2024-01-21
4,https://www.linkedin.com/jobs/view/group-unit-...,group/unit supervisor (systems support manager...,"IRS, Office of Chief Counsel","Chamblee, GA",2024-01-17,Gadsden,United States,Supervisor Travel-Information Center,Mid senior,Onsite,09:45:09.215838,2024-01-19


In [6]:
country_list = linkedin_job_postings['search_country'].unique().tolist()
country_list.sort()

In [7]:
linkedin_job_postings_by_country_dict = {}

In [8]:
for country in country_list:
    linkedin_job_postings_by_country_dict[country] = linkedin_job_postings[linkedin_job_postings['search_country']==country]

In [9]:
top = 10
top_cities_dict = {}

In [10]:
for country in country_list:
    top_cities_dict[country] = linkedin_job_postings_by_country_dict[country]['search_city'].value_counts().head(top)

In [11]:
fig_dict = {}

In [44]:
chart_colors = np.array(['rgb(239,85,59)', 'rgb(99,110,250)', 'rgb(254,203,82)', 'rgb(255,151,255)', 'rgb(182,232,128)',
                'rgb(255,102,146)', 'rgb(25,211,243)', 'rgb(255,161,90)', 'rgb(171,99,250)', 'rgb(0,204,150)'])

chart_colors_dict = {}

for country in country_list:
    chart_colors_dict[country] = chart_colors.copy()
    np.random.shuffle(chart_colors)

for country in country_list:
    total = top_cities_dict[country].values.sum()
    fig_dict[country] = go.Figure(
                            data=[
                                go.Pie(
                                    labels=top_cities_dict[country].index, 
                                    values=top_cities_dict[country].values,
                                    textinfo='label+value',
                                    insidetextorientation='horizontal',
                                    showlegend=False,
                                    pull=top_cities_dict[country].values / total,
                                    marker=dict(colors=chart_colors_dict[country]),
                                )
                            ]
                        )
    fig_dict[country].update_layout(
        title_text=f"Top {top} cities with highest jobs in {country}",
        title_x=0.5,
        title_y=0.95,
    )

In [45]:
fig = make_subplots(
    rows=2, 
    cols=2, 
    subplot_titles=[f"Top {top} cities with highest jobs in {country}" for country in country_list],
    specs=[[{"type": "sunburst"}, {"type": "sunburst"}], [{"type": "sunburst"}, {"type": "sunburst"}]],
    horizontal_spacing=0.2,
    vertical_spacing=0.1,
)
for country in country_list:
    fig.add_trace(fig_dict[country].data[0], row=country_list.index(country)//2+1, col=country_list.index(country)%2+1)

fig.update_layout(
    height=1200, 
    # width=1000, 
    title_text="Top 10 cities with highest jobs in each country",
)

In [ ]:
app = Dash()

In [ ]:
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

In [ ]:
Timer(1, open_browser).start()
app.run_server(debug=True, port=8080)

Error on request:
Traceback (most recent call last):
  File "c:\Users\ayush\AppData\Local\Programs\Python\Python311\Lib\site-packages\werkzeug\serving.py", line 362, in run_wsgi
    execute(self.server.app)
  File "c:\Users\ayush\AppData\Local\Programs\Python\Python311\Lib\site-packages\werkzeug\serving.py", line 347, in execute
    data = self.rfile.read(10_000_000)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
MemoryError
